### Importing Libraries

In [43]:
import pandas as pd
import numpy as np
import os, sys

### Importing the dataset

In [44]:
#data = pd.read_csv(r'Parkinsons Train Data.csv', header=0)
data = pd.read_csv(r'uci-parkinsons/parkinsons.data.csv', header=0)
data.head()

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,0.426,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,0.626,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,0.482,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,0.517,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,0.584,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


### Seperating the X and Y variables

In [45]:
features = data.loc[:, data.columns != 'status'].values[:,:]
labels = data.loc[:, 'status'].values

In [46]:
labels[labels == 1].shape[0], labels[labels == 0].shape[0]

(147, 48)

In [47]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler((-1, 1))
X = scaler.fit_transform(features)
Y = labels

In [49]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=7)

In [50]:
model = XGBClassifier()
model.fit(X_train, Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [51]:
Y_hat = [round(yhat) for yhat in model.predict(X_test)]
print(accuracy_score(Y_test, Y_hat)) # Test set accuracy
Y_hat = [round(yhat) for yhat in model.predict(X)]
print(accuracy_score(Y, Y_hat)) # Full set accuracy

0.9152542372881356
0.9743589743589743


In [8]:
X = data.loc[:, data.columns != 'status']
y = data.loc[:, data.columns == 'status']
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
os = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns
os_data_X,os_data_y=os.fit_sample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['status'])
# we can Check the numbers of our data
print("length of oversampled data is ",len(os_data_X))
print("Number of no subscription in oversampled data",len(os_data_y[os_data_y['status']==0]))
print("Number of subscription",len(os_data_y[os_data_y['status']==1]))
print("Proportion of no subscription data in oversampled data is ",len(os_data_y[os_data_y['status']==0])/len(os_data_X))
print("Proportion of subscription data in oversampled data is ",len(os_data_y[os_data_y['status']==1])/len(os_data_X))

length of oversampled data is  202
Number of no subscription in oversampled data 101
Number of subscription 101
Proportion of no subscription data in oversampled data is  0.5
Proportion of subscription data in oversampled data is  0.5


C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [103]:
print(data.columns.values)

['MDVP:Fo(Hz)' 'MDVP:Fhi(Hz)' 'MDVP:Flo(Hz)' 'MDVP:Jitter(%)'
 'MDVP:Jitter(Abs)' 'MDVP:RAP' 'MDVP:PPQ' 'Jitter:DDP' 'MDVP:Shimmer'
 'MDVP:Shimmer(dB)' 'Shimmer:APQ3' 'Shimmer:APQ5' 'MDVP:APQ' 'Shimmer:DDA'
 'NHR' 'HNR' 'status' 'RPDE' 'DFA' 'spread1' 'spread2' 'D2' 'PPE']


In [19]:
X = os_data_X[entire_cols]
y = os_data_y['status']
print(X.shape)
print("--------")
print(y.shape)

X = X.values.reshape(X.shape[1:])

(202, 22)
--------
(202,)


TypeError: 'numpy.ndarray' object is not callable

### Feature Selection

In [6]:
#analyzing the variables that won't affect the model
from sklearn import datasets
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

xgBoost = XGBClassifier()

rfe = RFE(xgBoost)
rfe = rfe.fit(features, labels)
print(rfe.support_)
print(rfe.ranking_)

[False  True  True  True False  True False False  True  True False  True
 False  True  True False  True  True  True False False False False  True
 False False]
[ 2  1  1  1 10  1 11  8  1  1 14  1 12  1  1  3  1  1  1  7  5  9  4  1
 13  6]


### Function to calculate XGBoost

In [13]:
def getXGBoost(X, y):
    from xgboost import XGBClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state=66, test_size = 0.2, stratify = data['status'])
    # Train
    model = XGBClassifier()
    model.fit(X_train, Y_train)
    
    # Evaluate
    Y_hat = [round(yhat) for yhat in model.predict(X_test)]
    print("Accuracy for test set -> ", end = "")
    print(accuracy_score(Y_test, Y_hat)) # Test set accuracy
    print("Accuracy for training set -> ", end = "")
    Y_hat = [round(yhat) for yhat in model.predict(X)]
    print(accuracy_score(y, Y_hat)) # Full set accuracy
    
    #Validations
    y_pred = model.predict(X_test)
    
    #Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(Y_test, y_pred) # Calulate Confusion matrix for test set.
    print(cm)

In [9]:
entire_cols = ['MDVP:Fo(Hz)', 'MDVP:Fhi(Hz)', 'MDVP:Flo(Hz)', 'MDVP:Jitter(%)',
 'MDVP:Jitter(Abs)', 'MDVP:RAP', 'MDVP:PPQ', 'Jitter:DDP', 'MDVP:Shimmer',
 'MDVP:Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5', 'MDVP:APQ', 'Shimmer:DDA',
 'NHR', 'HNR', 'RPDE', 'DFA', 'spread1', 'spread2', 'D2', 'PPE']
X = os_data_X[entire_cols]
y = os_data_y['status']

### Training the model with all the features

In [14]:
getXGBoost(X,y)

ValueError: Found input variables with inconsistent numbers of samples: [202, 195]

### Training the model with RFE features

In [82]:
y = data['Class information']
cols_RFE = ['Jitter(local, absolute)', 'Jitter (rap)', 'Jitter (ppq5)', 'Shimmer (local)',
        'Shimmer (apq5)', 'Shimmer (apq11)', 'AC', 'HTM', 'Median Pitch', 'Standard deviation',
        'Minimum pitch' , 'Maximum pitch', 'Fraction of locally unvoiced frames' ]
X = data[cols_RFE]
getXGBoost(X, y)

KeyError: 'Class information'

### Training and finding the best models (Features chosen pairwise)

In [10]:
y = data['Class information']
cols_1 = ['Jitter(local)','Shimmer (local)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_1]

In [11]:
getXGBoost(X, y)

Accuracy for test set -> 0.6923076923076923
Accuracy for training set -> 0.8519230769230769
[[76 28]
 [36 68]]


In [12]:
cols_2 = ['Jitter(local)','Shimmer (local, db)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_2]

In [13]:
getXGBoost(X, y)

Accuracy for test set -> 0.6875
Accuracy for training set -> 0.8442307692307692
[[76 28]
 [37 67]]


In [14]:
cols_3 = ['Jitter(local)','Shimmer (apq3)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_3]

In [15]:
getXGBoost(X, y)

Accuracy for test set -> 0.6682692307692307
Accuracy for training set -> 0.8432692307692308
[[73 31]
 [38 66]]


In [16]:
cols_4 = ['Jitter(local)','Shimmer (apq5)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_4]

In [17]:
getXGBoost(X, y)

Accuracy for test set -> 0.7067307692307693
Accuracy for training set -> 0.8557692307692307
[[77 27]
 [34 70]]


In [18]:
cols_5 = ['Jitter(local)','Shimmer (apq11)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_5]

In [19]:
getXGBoost(X, y)

Accuracy for test set -> 0.6971153846153846
Accuracy for training set -> 0.8451923076923077
[[73 31]
 [32 72]]


In [20]:
cols_6 = ['Jitter(local)','Shimmer (dda)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_6]

In [21]:
getXGBoost(X, y)

Accuracy for test set -> 0.6682692307692307
Accuracy for training set -> 0.8432692307692308
[[73 31]
 [38 66]]


In [22]:
cols_7 = ['Jitter(local, absolute)','Shimmer (local)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_7]

In [23]:
getXGBoost(X, y)

Accuracy for test set -> 0.6971153846153846
Accuracy for training set -> 0.8471153846153846
[[77 27]
 [36 68]]


In [24]:
cols_8 = ['Jitter(local, absolute)','Shimmer (local, db)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_8]

In [25]:
getXGBoost(X, y)

Accuracy for test set -> 0.6875
Accuracy for training set -> 0.8480769230769231
[[76 28]
 [37 67]]


In [26]:
cols_9 = ['Jitter(local, absolute)','Shimmer (apq3)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_9]

In [27]:
getXGBoost(X, y)

Accuracy for test set -> 0.6634615384615384
Accuracy for training set -> 0.8442307692307692
[[74 30]
 [40 64]]


In [28]:
cols_10 = ['Jitter(local, absolute)','Shimmer (apq5)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_10]

In [29]:
getXGBoost(X, y)

Accuracy for test set -> 0.6971153846153846
Accuracy for training set -> 0.8442307692307692
[[79 25]
 [38 66]]


In [30]:
cols_11 = ['Jitter(local, absolute)','Shimmer (apq11)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_11]

In [31]:
getXGBoost(X, y)

Accuracy for test set -> 0.7019230769230769
Accuracy for training set -> 0.8461538461538461
[[73 31]
 [31 73]]


In [32]:
cols_12 = ['Jitter(local, absolute)','Shimmer (dda)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_12]

In [33]:
getXGBoost(X, y)

Accuracy for test set -> 0.6634615384615384
Accuracy for training set -> 0.8442307692307692
[[74 30]
 [40 64]]


In [34]:
cols_13 = ['Jitter (rap)','Shimmer (local)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_13]

In [35]:
getXGBoost(X, y)

Accuracy for test set -> 0.6778846153846154
Accuracy for training set -> 0.8336538461538462
[[74 30]
 [37 67]]


In [36]:
cols_14 = ['Jitter (rap)','Shimmer (local, db)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_14]

In [37]:
getXGBoost(X, y)

Accuracy for test set -> 0.6586538461538461
Accuracy for training set -> 0.8326923076923077
[[74 30]
 [41 63]]


In [38]:
cols_15 = ['Jitter (rap)','Shimmer (apq3)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_15]

In [39]:
getXGBoost(X, y)

Accuracy for test set -> 0.6923076923076923
Accuracy for training set -> 0.8336538461538462
[[78 26]
 [38 66]]


In [40]:
cols_16 = ['Jitter (rap)','Shimmer (apq5)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_16]

In [41]:
getXGBoost(X, y)

Accuracy for test set -> 0.7067307692307693
Accuracy for training set -> 0.8490384615384615
[[81 23]
 [38 66]]


In [42]:
cols_17 = ['Jitter (rap)','Shimmer (apq11)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_17]

In [43]:
getXGBoost(X, y)

Accuracy for test set -> 0.6826923076923077
Accuracy for training set -> 0.8336538461538462
[[74 30]
 [36 68]]


In [44]:
cols_18 = ['Jitter (rap)','Shimmer (dda)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_18]

In [45]:
getXGBoost(X, y)

Accuracy for test set -> 0.6923076923076923
Accuracy for training set -> 0.8336538461538462
[[78 26]
 [38 66]]


In [46]:
cols_19 = ['Jitter (ppq5)','Shimmer (local)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_19]

In [47]:
getXGBoost(X, y)

Accuracy for test set -> 0.7259615384615384
Accuracy for training set -> 0.8365384615384616
[[77 27]
 [30 74]]


In [48]:
cols_20 = ['Jitter (ppq5)','Shimmer (local, db)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_20]

In [49]:
getXGBoost(X, y)

Accuracy for test set -> 0.6923076923076923
Accuracy for training set -> 0.8365384615384616
[[79 25]
 [39 65]]


In [50]:
cols_21 = ['Jitter (ppq5)','Shimmer (apq3)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_21]

In [51]:
getXGBoost(X, y)

Accuracy for test set -> 0.7307692307692307
Accuracy for training set -> 0.8538461538461538
[[80 24]
 [32 72]]


In [52]:
cols_22 = ['Jitter (ppq5)','Shimmer (apq5)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_22]

In [53]:
getXGBoost(X, y)

Accuracy for test set -> 0.7403846153846154
Accuracy for training set -> 0.8586538461538461
[[78 26]
 [28 76]]


In [54]:
cols_23 = ['Jitter (ppq5)','Shimmer (apq11)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_23]

In [55]:
getXGBoost(X, y)

Accuracy for test set -> 0.7115384615384616
Accuracy for training set -> 0.8538461538461538
[[75 29]
 [31 73]]


In [56]:
cols_24 = ['Jitter (ppq5)','Shimmer (dda)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_24]

In [57]:
getXGBoost(X, y)

Accuracy for test set -> 0.7307692307692307
Accuracy for training set -> 0.8538461538461538
[[80 24]
 [32 72]]


In [58]:
cols_25 = ['Jitter (ddp)','Shimmer (local)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_25]
y = data['Class information']

In [59]:
getXGBoost(X, y)

Accuracy for test set -> 0.6875
Accuracy for training set -> 0.8432692307692308
[[77 27]
 [38 66]]


In [60]:
cols_26 = ['Jitter (ddp)','Shimmer (local, db)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_26]

In [61]:
getXGBoost(X, y)

Accuracy for test set -> 0.6826923076923077
Accuracy for training set -> 0.8365384615384616
[[75 29]
 [37 67]]


In [62]:
cols_27 = ['Jitter (ddp)','Shimmer (apq3)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_27]

In [63]:
getXGBoost(X, y)

Accuracy for test set -> 0.6730769230769231
Accuracy for training set -> 0.8384615384615385
[[79 25]
 [43 61]]


In [64]:
cols_28 = ['Jitter (ddp)','Shimmer (apq5)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_28]

In [65]:
getXGBoost(X, y)

Accuracy for test set -> 0.7019230769230769
Accuracy for training set -> 0.8480769230769231
[[82 22]
 [40 64]]


In [66]:
cols_29 = ['Jitter (ddp)','Shimmer (apq11)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_29]

In [67]:
getXGBoost(X, y)

Accuracy for test set -> 0.7019230769230769
Accuracy for training set -> 0.8442307692307692
[[75 29]
 [33 71]]


In [68]:
cols_30 = ['Jitter (ddp)','Shimmer (dda)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
X = data[cols_30]
y = data['Class information']

In [69]:
getXGBoost(X, y)

Accuracy for test set -> 0.6730769230769231
Accuracy for training set -> 0.8384615384615385
[[79 25]
 [43 61]]
